In [1]:
import sys
sys.path.append('/scratch/pradap/python-work/dmagellan')
import py_entitymatching as em
import os
import time
import psutil
import pandas as pd

from dask import get, multiprocessing
from dmagellan.feature.extractfeatures import extract_feature_vecs
from dmagellan.feature.autofeaturegen import get_features_for_matching
from dmagellan.matcher.dtmatcher import DTMatcher
from dmagellan.matcher.svmmatcher import SVMMatcher
from dmagellan.matcher.rfmatcher import RFMatcher
from dmagellan.matcher.logregmatcher import LogRegMatcher
from dmagellan.matcher.nbmatcher import NBMatcher
from dmagellan.matcher.linregmatcher import LinRegMatcher

from dmagellan.mlmatcherselection.mlmatcherselection import select_matcher

In [2]:
print("Mem. usage before reading:{0}".format( psutil.virtual_memory().used/1e9))
path_feat_vecs = '../datasets/featvecs_citeseer.csv'
H = pd.read_csv(path_feat_vecs)
H.fillna(value=0, inplace=True)
print("Mem. usage after reading:{0}".format(psutil.virtual_memory().used/1e9))

Mem. usage before reading:1.445629952
Mem. usage after reading:1.44531456


In [3]:
memUsageBefore = psutil.virtual_memory().used/1e9
timeBefore = time.time()
# Create a set of ML-matchers
dt = DTMatcher(name='DecisionTree', random_state=0)
svm = SVMMatcher(name='SVM', random_state=0)
rf = RFMatcher(name='RF', random_state=0)
lg = LogRegMatcher(name='LogReg', random_state=0)
nb = NBMatcher(name='NaiveBayes')
ln = LinRegMatcher(name='LinearRegression')

In [4]:
len(H)

500

In [5]:
result = select_matcher([dt, lg, svm, rf, nb, ln], table=H,
        exclude_attrs=['_id', 'l_id', 'r_id', 'label'],
        k=len(H)-1,
        target_attr='label', metric='f1', random_state=0, compute=False)

In [6]:
# result.visualize()

In [7]:
D = result.compute(get=multiprocessing.get)

/scratch/pradap/local/share/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/scratch/pradap/local/share/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/scratch/pradap/local/share/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/scratch/pradap/local/share/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [8]:
D

OrderedDict([('selected_matcher',
              <dmagellan.matcher.nbmatcher.NBMatcher at 0x7fb13a380090>),
             ('cv_stats',                Name  \
              0      DecisionTree   
              1            LogReg   
              2               SVM   
              3                RF   
              4        NaiveBayes   
              5  LinearRegression   
              
                                                                                  Matcher  \
              0          <dmagellan.matcher.dtmatcher.DTMatcher object at 0x7fb13a38e950>   
              1  <dmagellan.matcher.logregmatcher.LogRegMatcher object at 0x7fb13a39b110>   
              2        <dmagellan.matcher.svmmatcher.SVMMatcher object at 0x7fb13a39b290>   
              3          <dmagellan.matcher.rfmatcher.RFMatcher object at 0x7fb13a39bd10>   
              4          <dmagellan.matcher.nbmatcher.NBMatcher object at 0x7fb13a380090>   
              5  <dmagellan.matcher.linregmatche